In [22]:
import torch
import numpy as np

In [24]:
data_dict = torch.load('data/uci_data/uci_heart_torch.pt')

In [26]:
data_dict.keys()

dict_keys(['train', 'val', 'iid_test', 'ood_test'])

In [35]:
tmp = data_dict['train']

In [36]:
tmp = list(zip(*tmp))

In [ ]:
X = torch.stack(tmp[0])
y = torch.tensor(tmp[1])


In [52]:
min_ = torch.min(X, dim=0).values
max_ = torch.max(X, dim=0).values

In [54]:
X = (X-min_)/(max_ - min_)

In [56]:
from experiments.utils import TensorDataset

In [ ]:
class Test(TensorDataset):
    def hi(self):
        return 3
    def __str__(self):
        return 3

In [62]:
x = Test()

TypeError: TensorDataset.__init__() missing 2 required positional arguments: 'X' and 'y'